## Import DrugBank-SIDER overlap 

In [1]:
# Use DB-SIDER overlap file to make file to output with drug-target and then drug-se for this smaller list 

In [2]:
import pandas as pd
import numpy as np

In [3]:
# import data 
sider_db_overlap = pd.read_csv("~/Desktop/Harvard/Thesis/Data/SIDER/smiles/sider_drugbank_smiles_overlap.txt", 
                    sep='\t')

In [4]:
sider_db_overlap.head()

,pubchem_id,SMILES[1],SMILES[2],Canonical_Smiles,DrugID,Name,Status,Type
0,10917,C[N+](C)(C)C[C@@H](CC(=O)[O-])O,C[N+](C)(C)C[C@H](O)CC([O-])=O,C[N+](C)(C)C[C@H](O)CC(=O)[O-],DB00583,L-Carnitine,approved,small molecule
1,119,C(CC(=O)O)CN,NCCCC(O)=O,NCCCC(=O)O,DB02530,Gamma(Amino)-Butyric Acid,experimental,small molecule
2,137,C(CC(=O)O)C(=O)CN,NCC(=O)CCC(O)=O,NCC(=O)CCC(=O)O,DB00855,Aminolevulinic acid,approved,small molecule
3,444412,CN1[C@H](CNC2=C1C(=O)N=C(N2)N)CNC3=CC=C(C=C3)C...,CN1[C@@H](CNC2=CC=C(C=C2)C(=O)N[C@@H](CCC(O)=O...,CN1[C@@H](CNc2ccc(cc2)C(=O)N[C@@H](CCC(=O)O)C(...,DB04789,5-methyltetrahydrofolate,experimental,small molecule
4,5280360,CCCCC[C@@H](/C=C/[C@H]1[C@@H](CC(=O)[C@@H]1C/C...,CCCCC[C@H](O)\C=C\[C@H]1[C@H](O)CC(=O)[C@@H]1C...,CCCCC[C@H](O)\C=C\[C@H]1[C@H](O)CC(=O)[C@@H]1C...,DB00917,Dinoprostone,approved,small molecule


In [5]:
sider_db_overlap = sider_db_overlap.drop(['SMILES[1]', 'SMILES[2]', 'Canonical_Smiles', 'Status', 'Type'], axis = 1)

In [6]:
sider_db_overlap.head()

,pubchem_id,DrugID,Name
0,10917,DB00583,L-Carnitine
1,119,DB02530,Gamma(Amino)-Butyric Acid
2,137,DB00855,Aminolevulinic acid
3,444412,DB04789,5-methyltetrahydrofolate
4,5280360,DB00917,Dinoprostone


## Import DrugBank data 

In [7]:
## import DrugBank data
interactions = pd.read_table("~/Desktop/Harvard/Thesis/Data/DrugBank/interactions.txt")
targets = pd.read_table("~/Desktop/Harvard/Thesis/Data/DrugBank/targets.txt")

In [8]:
interactions.head()

,DrugID,TargetID,Action
0,DB00001,BE0000048,inhibitor
1,DB00002,BE0000767,antagonist
2,DB00002,BE0000901,NaN
3,DB00002,BE0002093,NaN
4,DB00002,BE0002094,NaN


In [14]:
print(len(interactions), '  unique drugIDs:',len(interactions.DrugID.unique()), 
    '  unique Targets:' , len(interactions.TargetID.unique()))

15709   unique drugIDs: 6835   unique Targets: 4217


In [15]:
## interactions is not 1:1 
# there are more drugs than targets but a drug can have several targets 

In [21]:
targets.head(15)

,TargetID,UniProt,HGNC,Species,Name
0,BE0000048,P00734,HGNC:3535,9606.0,Prothrombin
1,BE0000767,P00533,HGNC:3236,9606.0,Epidermal growth factor receptor
2,BE0000901,O75015,HGNC:3620,9606.0,Low affinity immunoglobulin gamma Fc region re...
3,BE0002093,P00736,HGNC:1246,9606.0,Complement C1r subcomponent
4,BE0002094,P02745,HGNC:1241,9606.0,Complement C1q subcomponent subunit A
5,BE0002095,P02746,HGNC:1242,9606.0,Complement C1q subcomponent subunit B
6,BE0002096,P02747,HGNC:1245,9606.0,Complement C1q subcomponent subunit C
7,BE0002097,P08637,HGNC:3619,9606.0,Low affinity immunoglobulin gamma Fc region re...
8,BE0001529,P09871,HGNC:1247,9606.0,Complement C1s subcomponent
9,BE0000710,P12314,HGNC:3613,9606.0,High affinity immunoglobulin gamma Fc receptor I


In [22]:
print(len(targets), len(targets.TargetID.unique()), len(targets.UniProt.unique()))

4217 4217 4102


In [26]:
## There is one target and not all targets have uniprot IDs 
# get rid of targets without uniprot ID 
# 115 targets dont have uniprot IDs 
targets = targets[targets['UniProt'].notnull()]

In [27]:
targets.head(15)

,TargetID,UniProt,HGNC,Species,Name
0,BE0000048,P00734,HGNC:3535,9606.0,Prothrombin
1,BE0000767,P00533,HGNC:3236,9606.0,Epidermal growth factor receptor
2,BE0000901,O75015,HGNC:3620,9606.0,Low affinity immunoglobulin gamma Fc region re...
3,BE0002093,P00736,HGNC:1246,9606.0,Complement C1r subcomponent
4,BE0002094,P02745,HGNC:1241,9606.0,Complement C1q subcomponent subunit A
5,BE0002095,P02746,HGNC:1242,9606.0,Complement C1q subcomponent subunit B
6,BE0002096,P02747,HGNC:1245,9606.0,Complement C1q subcomponent subunit C
7,BE0002097,P08637,HGNC:3619,9606.0,Low affinity immunoglobulin gamma Fc region re...
8,BE0001529,P09871,HGNC:1247,9606.0,Complement C1s subcomponent
9,BE0000710,P12314,HGNC:3613,9606.0,High affinity immunoglobulin gamma Fc receptor I


In [29]:
interactions = interactions.drop(['Action'], axis = 1)

In [30]:
interactions.head()

,DrugID,TargetID
0,DB00001,BE0000048
1,DB00002,BE0000767
2,DB00002,BE0000901
3,DB00002,BE0002093
4,DB00002,BE0002094


In [39]:
# maps targetID to uniprot id (1:1 relationship)
target_uniprot_dict = (targets.drop(['HGNC', 'Species', 'Name'], axis = 1)).set_index('TargetID')['UniProt'].to_dict()

# use for loop to subset df for each unique drug ID and then for that subset, grab targets
# maps drugid to list of targets (targetID), value is a list of targets
drug_target_dict = {} 
drug_ids = list(interactions.DrugID.unique())
for drugID in drug_ids:
    subset = interactions.loc[interactions['DrugID'] == drugID]
    target_list = list(subset.TargetID)
    drug_target_dict[drugID] = target_list

In [43]:
sider_db_overlap.head()

,pubchem_id,DrugID,Name
0,10917,DB00583,L-Carnitine
1,119,DB02530,Gamma(Amino)-Butyric Acid
2,137,DB00855,Aminolevulinic acid
3,444412,DB04789,5-methyltetrahydrofolate
4,5280360,DB00917,Dinoprostone


In [50]:
## create new dataframe with pubchem ID and target
pubchem_ids = []
targets = []
count = 0
count1 = 0
for index, row in sider_db_overlap.iterrows():
    if row[1] in drug_target_dict.keys():
        for target in drug_target_dict[row[1]]:
            if target in target_uniprot_dict.keys():
                pubchem_ids.append(row[0])
                targets.append(target_uniprot_dict[target])
                continue 
            else:
                #print('target', target, 'not in target_uniprot_dict')
                count += 1
    else:
        #print('drug id' , row[1], ' not in drug_target_dict')
        count1 += 1 
print(count, 'targets do not have uniprot counterparts')
print(count1, 'drugs to now have targets in drugbank')

104 targets do not have uniprot counterparts
37 drugs to now have targets in drugbank


In [54]:
print(len(pubchem_ids), len(targets))
print(len(set(pubchem_ids)), len(set(targets)))

3018 3018
755 882


In [55]:
drug_target_pairings = pd.DataFrame(
    {'pubchem_id': pubchem_ids,
     'uniprot_target': targets
    })

In [56]:
drug_target_pairings.head()

,pubchem_id,uniprot_target
0,10917,Q9H015
1,10917,O76082
2,10917,P43155
3,10917,Q8N8R3
4,10917,O43772


In [60]:
drug_target_pairings.to_csv("~/Desktop/Harvard/Thesis/Data/SIDER/smiles/drug_target_pairings.txt",
                            sep = '\t', index = False)